Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 1

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Choose which observations you will use to train, validate, and test your model. And which observations, if any, to exclude.
- [ ] Determine whether your problem is regression or classification.
- [ ] Choose your evaluation metric.
- [ ] Begin with baselines: majority class baseline for classification, or mean baseline for regression, with your metric of choice.
- [ ] Begin to clean and explore your data.
- [ ] Choose which features, if any, to exclude. Would some features "leak" information from the future?

## Reading
- [Attacking discrimination with smarter machine learning](https://research.google.com/bigpicture/attacking-discrimination-in-ml/), by Google Research, with  interactive visualizations. _"A threshold classifier essentially makes a yes/no decision, putting things in one category or another. We look at how these classifiers work, ways they can potentially be unfair, and how you might turn an unfair classifier into a fairer one. As an illustrative example, we focus on loan granting scenarios where a bank may grant or deny a loan based on a single, automatically computed number such as a credit score."_
- [How Shopify Capital Uses Quantile Regression To Help Merchants Succeed](https://engineering.shopify.com/blogs/engineering/how-shopify-uses-machine-learning-to-help-our-merchants-grow-their-business)
- [Maximizing Scarce Maintenance Resources with Data: Applying predictive modeling, precision at k, and clustering to optimize impact](https://towardsdatascience.com/maximizing-scarce-maintenance-resources-with-data-8f3491133050), **by Lambda DS3 student** Michael Brady. His blog post extends the Tanzania Waterpumps scenario, far beyond what's in the lecture notebook.
- [Notebook about how to calculate expected value from a confusion matrix by treating it as a cost-benefit matrix](https://github.com/podopie/DAT18NYC/blob/master/classes/13-expected_value_cost_benefit_analysis.ipynb)
- [Simple guide to confusion matrix terminology](https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/) by Kevin Markham, with video
- [Visualizing Machine Learning Thresholds to Make Better Business Decisions](https://blog.insightdatascience.com/visualizing-machine-learning-thresholds-to-make-better-business-decisions-4ab07f823415)

In [12]:
################################ EDA IMPORTS ###################################
%matplotlib inline
import pandas as pd 
import pandas_profiling # Quick Data Analysis
import numpy as np # Linear Algebra lib
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objs as go # interactive low-level plotting lib https://plot.ly/python/
import plotly.express as px #high-level api wrapper for plotly https://plot.ly/python/plotly-express/#visualize-distributions
# ---------------- Plot libs settings ------------- #
# Pick style of Matplolib plots 
# Different style sheets:-> https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
# Configure Seaborn Asthetics: -> https://seaborn.pydata.org/tutorial/aesthetics.html?highlight=style
plt.style.use('seaborn-darkgrid')
sns.set(context='notebook', style='darkgrid', palette='colorblind')
# Seting a universal figure size 
plt.rcParams['figure.figsize'] = (10, 6)

# ---------------- Pandas settings ------------- #
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

################################################################################

## Project PUBG 🤖💣💥🔫🤖 


### So, where we droppin' boys and girls?
PUBG or Player Unknown Battlegrounds is a battle-royale style game where at the beginning of the play, nearly 100 people parachute onto an island without any equipment. In order to win the game, you need to scavenge for weapons and available equipment to eliminate the other people and survive to the end. The game also restricts player in Hunger Game style by reducing the playable are in map after a some amount of fixed time is passed. 

### Objective

**Defining the problem:** The problem we have is, there is not set guide or strategy to improve player performance in PUBG, should your play style be stealth like a ninja and sneak upon unsuspecting players, or by camping in one spot and hide your way into victory, or snipe like assassin, or do you need to be aggressive and play like Rambo? 

**Solution:** Is to create a web application that allows players to improve their strategy by considering *What-if* scenarios as MVP.

### Data Mining

The team at [PUBG](https://www.pubg.com/) has made official game data available for the public to explore and scavenge outside of "The Blue Circle." This competition is not an official or affiliated PUBG site - Kaggle collected data made possible through the [PUBG Developer API](https://developer.pubg.com/).

![](https://d.newsweek.com/en/full/854048/pubg-logo.jpg)

### Data Description

We are given over 65,000 games worth of anonymized player data, split into training and testing sets. 

- DBNOs - Number of enemy players knocked.
-assists - Number of enemy players this player damaged that were killed by teammates.
- boosts - Number of boost items used.
- damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
- headshotKills - Number of enemy players killed with headshots.
- heals - Number of healing items used.
- Id - Player’s Id
- killPlace - Ranking in match of number of enemy players killed.
- killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.
- killStreaks - Max number of enemy players killed in a short amount of time.
- kills - Number of enemy players killed.
- longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
- matchDuration - Duration of match in seconds.
- matchId - ID to identify match. There are no matches that are in both the training and testing set.
- matchType - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.
- rankPoints - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.
- revives - Number of times this player revived teammates.
- rideDistance - Total distance traveled in vehicles measured in meters.
- roadKills - Number of kills while in a vehicle.
- swimDistance - Total distance traveled by swimming measured in meters.
- teamKills - Number of times this player killed a teammate.
- vehicleDestroys - Number of vehicles destroyed.
- walkDistance - Total distance traveled on foot measured in meters.
- weaponsAcquired - Number of weapons picked up.
- winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.
- groupId - ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
- numGroups - Number of groups we have data for in the match.
- maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
- winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

### Load Dataset

In [2]:
def reduce_mem_usage(df, verbose=True):
    """ Function iterates through all the columns of a dataframe and modify the data type
        to reduce memory usage.
        
        Credit to: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
        
        Parameters
        ----------
        df : Pandas DataFrame
        verbose: (True) by default, prints out before and after memory usage
        
        Returns
        -------
        df : Reduced Memory Pandas DataFrame
        
    """
    
    if verbose:
        start_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    if verbose:
        end_mem = df.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df



def load_dataset(verbose=True, develop=True):
    
    # If you're in Colab...
    import os, sys
    in_colab = 'google.colab' in sys.modules
    if in_colab:
    # Install required python package:
    # category_encoders, version >= 2.0
        !pip install --upgrade category_encoders
        
        # Pull files from Github repo
        os.chdir('/content')
        !git init .
        !git remote add origin https://github.com/hurshd0/DS-Unit-2-Applied-Modeling.git
        !git pull origin master
        
        # Change into directory for module
        os.chdir('module1')
        
        # Download ~1GB dataset
        !rm -rf ../data/project-pubg
        !mkdir -p ../data/project-pubg
        !wget 'https://project-pubg.s3.us-east-2.amazonaws.com/train_V2.csv' -P ../data/project-pubg
        !wget 'https://project-pubg.s3.us-east-2.amazonaws.com/test_V2.csv' -P ../data/project-pubg
        !wget 'https://project-pubg.s3.us-east-2.amazonaws.com/sample_submission_V2.csv' -P ../data/project-pubg
    
    
    # Load training set
    if verbose:
        print('-' * 80)
        print('Load: train')
    train = pd.read_csv('../data/project-pubg/train_V2.csv')
    train = reduce_mem_usage(train, verbose=verbose)
    
    # Load sample submission
    if not develop:
        # Load test set
        if verbose:
            print('-' * 80)
            print('Load: test')
        test = pd.read_csv('../data/project-pubg/test_V2.csv')
        test = reduce_mem_usage(test, verbose=verbose)
    
        if verbose:
            print('-' * 80)
            print('Load: sample submission')
        sample_submission = pd.read_csv('../data/project-pubg/sample_submission_V2.csv')
        sample_submission = reduce_mem_usage(sample_submission, verbose=verbose)
        
        if verbose:
            print(
            f'''
            -------------------- SHAPE ---------------------
            Train Set: {train.shape}
            Test Set: {test.shape}
            Sample Submission: {sample_submission.shape}
            ------------------------------------------------
            ''')
        # Return data frames
        return (train, test, sample_submission)
    
    return train

traindf = load_dataset()

--------------------------------------------------------------------------------
Load: train
Memory usage of dataframe is 983.90 MB
Memory usage after optimization is: 288.39 MB
Decreased by 70.7%


In [3]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int8
boosts             int8
damageDealt        float16
DBNOs              int8
headshotKills      int8
heals              int8
killPlace          int8
killPoints         int16
kills              int8
killStreaks        int8
longestKill        float16
matchDuration      int16
matchType          object
maxPlace           int8
numGroups          int8
rankPoints         int16
revives            int8
rideDistance       float16
roadKills          int8
swimDistance       float16
teamKills          int8
vehicleDestroys    int8
walkDistance       float16
weaponsAcquired    int16
winPoints          int16
winPlacePerc       float16
dtypes: float16(6), int16(5), int8(14), object(4)
memory usage: 288.4+ MB


### Check for Missing Values

In [5]:
traindf.isnull().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       1
dtype: int64

Let's look at the row which has missing `target` value.

In [13]:
traindf[traindf['winPlacePerc'].isnull()]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
2744604,f70c74418bb064,12dfbede33f92b,224a123c53e008,0,0,0.0,0,0,0,1,0,0,0,0.0,9,solo-fpp,1,1,1574,0,0.0,0,0.0,0,0,0.0,0,0,NaN


Looks like most values are `0`, it is safe to drop this observation now.

In [14]:
traindf = traindf[~traindf['winPlacePerc'].isnull()]

### Check for duplicates

In [18]:
traindf[traindf.duplicated()]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc


Great! there are no duplicates.

## Assignment

- [X] Choose your target. Which column in your tabular dataset will you predict?

`target` in this case is `winPlacePerc`, column, that predicts percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match.

- [X] Choose which observations you will use to train, validate, and test your model. And which observations, if any, to exclude.

Since this is dataset was obtained from `Kaggle` competition, we would just need to split the `training` set into `train` and `validation` set. Our strategy would be to:

- Use **Train/validate split: random 80/20%** for 
- **K-fold Cross-Validation**

For now let's use `80/20 split`. 

- [X] Determine whether your problem is regression or classification.

Based on the problem statement, our problem is framed as **regression** type, but we can also `bin` the winning percentiles into groups and predict those groups instead. 

- [X] Choose your evaluation metric.

Since we are dealing with **regression** type question, and target is `percentile winning placement` between 0 and 1, we can use MAE, RMSE, and $R^2$ to better understand the model.

- [X] Begin with baselines: majority class baseline for classification, or mean baseline for regression, with your metric of choice.

In [31]:
traindf['winPlacePerc'].describe()

count    4.446965e+06
mean              NaN
std      0.000000e+00
min      0.000000e+00
25%      1.999512e-01
50%      4.582520e-01
75%      7.407227e-01
max      1.000000e+00
Name: winPlacePerc, dtype: float64

- [X] Begin to clean and explore your data.

- [X] Choose which features, if any, to exclude. Would some features "leak" information from the future?